## SQL!!!!!

How much to you remember?


In [2]:
import sqlite3
import pandas as pd

con = sqlite3.connect('./data/flights.db')

Remember the schema.

In [3]:
schema_df = pd.read_sql("""

SELECT *
FROM sqlite_master


""", con)

schema_df

,type,name,tbl_name,rootpage,sql
0,table,airports,airports,2,"CREATE TABLE airports (\n[index] INTEGER,\n [..."
1,index,ix_airports_index,airports,3,CREATE INDEX ix_airports_index ON airports ([i...
2,table,airlines,airlines,945,"CREATE TABLE airlines (\n[index] INTEGER,\n [..."
3,index,ix_airlines_index,airlines,946,CREATE INDEX ix_airlines_index ON airlines ([i...
4,table,routes,routes,1393,"CREATE TABLE routes (\n[index] INTEGER,\n [ai..."
5,index,ix_routes_index,routes,1394,CREATE INDEX ix_routes_index ON routes ([index])


Let's get a list of all the table names

In [4]:
table_names = list(schema_df[ schema_df['type'] == 'table']['tbl_name'])

Now, let's step through that list and display one row from each table

In [5]:
for table in table_names:
    qry = """
    SELECT *
    FROM {}
    LIMIT 1
    """.format(table)
    
    print(f'First row for table {table}')
    display(pd.read_sql(qry, con))
    print('\n\n')

First row for table airports


,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby





First row for table airlines


,index,id,name,alias,iata,icao,callsign,country,active
0,0,1,Private flight,\N,-,None,None,None,Y





First row for table routes


,index,airline,airline_id,source,source_id,dest,dest_id,codeshare,stops,equipment
0,0,2B,410,AER,2965,KZN,2990,None,0,CR2


### Simple Query

Which country has the most airports? Which one has the least?

In [6]:
most_airports = pd.read_sql("""

SELECT 
    country,
    COUNT()
FROM 
    airports
GROUP BY
    COUNTRY
ORDER BY
    COUNT() DESC
LIMIT 1

""", con)
most_airports

,country,COUNT()
0,United States,1697


In [7]:
least_airports = pd.read_sql("""

SELECT 
    country,
    COUNT()
FROM 
    airports
GROUP BY
    COUNTRY
ORDER BY
    COUNT()
LIMIT 10
""", con)
most_airports

,country,COUNT()
0,United States,1697


### Where?

Which airports are in the `Pacific/Port_Moresby` time zone? What city and country?

### Further down the rabbit hole!

Which countries have between 50 and 100 airports?

### Joins!

What is the name of the airline has the most routes? The fewest?

### Subqueries/CTEs!

What is the average number of routes per airline?

In [9]:
# sub-query
pd.read_sql("""

SELECT 
    AVG(number_of_routes) AS average_routes

FROM (
    SELECT
        airline_id,
        COUNT(routes.airline_id) AS number_of_routes
    FROM
        routes
    GROUP BY
        airline_id
        )

""", con)

,average_routes
0,125.07024


In [11]:
# CTE
pd.read_sql("""
WITH avg_routes AS (
    SELECT
        airline, airline_id, COUNT() as num_routes
    FROM
        routes
    GROUP BY
        airline_id
)
SELECT
    AVG(num_routes) AS avg_num_routes
FROM
    avg_routes

""", con)

,avg_num_routes
0,125.07024


### Multiple Joins!

What is the name of the airline with the most departing flights in the `Pacific/Port_Moresby` timezone?

In [15]:
pd.read_sql("""

SELECT
    COUNT(rt.airline_id) AS rt_count,
    al.name as Name
FROM
    routes AS rt
    INNER JOIN airports AS ap ON rt.source_id = ap.id
    INNER JOIN airlines AS al on rt.airline_id = al.id
WHERE
    ap.timezone = 'Pacific/Port_Moresby'
GROUP BY
    rt.airline_id
ORDER BY
    rt_count DESC
LIMIT 1

""", con)

,rt_count,Name
0,82,Air Niugini
